In [4]:
import requests
from html.parser import HTMLParser
import json
import time

In [5]:
class MyHTMLParser(HTMLParser):
    def __init__(self,res_str=[]):
        super().__init__()
        self.res_str = res_str
        
    def handle_data(self, data):
        self.res_str.append(str(data))
    
    def get_text(self):
        return "".join(self.res_str)
    
    def clear(self):
        self.res_str.clear()

In [6]:
headers = {
    'Connection': "keep-alive",
    'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.183 Safari/537.36"
}

In [7]:
url = 'https://api.dayi.org.cn:9997/api/medical/getList?pageNo=%d&pageSize=10'
url2 = 'https://api.dayi.org.cn:9997/api/medical/%s'
total_page = 1896

In [8]:
parser = MyHTMLParser()
with open('yao_id.txt','w') as f1 \
    ,open('yao_err_page.txt','w') as f2 \
    ,open('yao.txt','w') as f3:
    for i in range(1507,total_page):
        try:
            f_url = url % i
            r = requests.get(f_url,headers=headers,timeout=10)

            parser.clear()
            parser.feed(r.text)
            res_txt = parser.get_text()

            json_txt = json.loads(res_txt)
            for item in json_txt['list']:
                yid = str(item['id']) + '\n'
                f1.write(yid)
                
                f_url2 = url2 % str(item['id'])
                medical = requests.get(f_url2,headers=headers,timeout=5)
                
                parser.clear()
                parser.feed(medical.text)
                res_txt = parser.get_text()
                f3.write(res_txt)
                f3.write("\n")
                
                time.sleep(3)
                
            if i % 10 == 0:
                f1.flush()
                f2.flush()
                f3.flush()
                
            time.sleep(3)
            #print('=====',i)
        except Exception as e:
            print('获取当前页数据异常：',i)
            err_page = str(i) + '\n'
            f2.write(err_page)
        
parser.close()

In [12]:
import pandas as pd

In [13]:
df1 = pd.read_csv('yao_id-Copy1.txt')
len(df1)

18950

In [14]:
df2 = df1.drop_duplicates()
len(df2)

18702

In [ ]:
df1